In [21]:
'''
Input arguments:

num_output: this value has nothing to do with the number of classes, batch_size, etc., 
and it is mostly equal to 1. If the network is a **multi-stream network** 
(forked network with multiple outputs), set the value to the number of outputs.

quantize: if set to True, use the quantize feature of Tensorflow
(https://www.tensorflow.org/performance/quantization) [default: False]

use_theano: Thaeno and Tensorflow implement convolution in different ways.
When using Keras with Theano backend, the order is set to 'channels_first'.
This feature is not fully tested, and doesn't work with quantizization [default: False]

input_fld: directory holding the keras weights file [default: .]

output_fld: destination directory to save the tensorflow files [default: .]

input_model_file: name of the input weight file [default: 'model.h5']

output_model_file: name of the output weight file [default: args.input_model_file + '.pb']

graph_def: if set to True, will write the graph definition as an ascii file [default: False]

output_graphdef_file: if graph_def is set to True, the file name of the 
graph definition [default: model.ascii]

output_node_prefix: the prefix to use for output nodes. [default: output_node]

'''

"\nInput arguments:\n\nnum_output: this value has nothing to do with the number of classes, batch_size, etc., \nand it is mostly equal to 1. If the network is a **multi-stream network** \n(forked network with multiple outputs), set the value to the number of outputs.\n\nquantize: if set to True, use the quantize feature of Tensorflow\n(https://www.tensorflow.org/performance/quantization) [default: False]\n\nuse_theano: Thaeno and Tensorflow implement convolution in different ways.\nWhen using Keras with Theano backend, the order is set to 'channels_first'.\nThis feature is not fully tested, and doesn't work with quantizization [default: False]\n\ninput_fld: directory holding the keras weights file [default: .]\n\noutput_fld: destination directory to save the tensorflow files [default: .]\n\ninput_model_file: name of the input weight file [default: 'model.h5']\n\noutput_model_file: name of the output weight file [default: args.input_model_file + '.pb']\n\ngraph_def: if set to True, will

Parse input arguments

In [22]:
import argparse
parser = argparse.ArgumentParser(description='set input arguments')
parser.add_argument('-input_fld', action="store", 
                    dest='input_fld', type=str, default='.')
parser.add_argument('-output_fld', action="store", 
                    dest='output_fld', type=str, default='')
parser.add_argument('-input_model_file', action="store", 
                    dest='input_model_file', type=str, default='weights_at_epoch_18.h5')
parser.add_argument('-output_model_file', action="store", 
                    dest='output_model_file', type=str, default='')
parser.add_argument('-output_graphdef_file', action="store", 
                    dest='output_graphdef_file', type=str, default='model.ascii') #read the model.ascii from 
parser.add_argument('-num_outputs', action="store", 
                    dest='num_outputs', type=int, default=1)
parser.add_argument('-graph_def', action="store", 
                    dest='graph_def', type=bool, default=True) #if set to true write graph as .asci
parser.add_argument('-output_node_prefix', action="store", 
                    dest='output_node_prefix', type=str, default='output_node')
parser.add_argument('-quantize', action="store", 
                    dest='quantize', type=bool, default=False)
parser.add_argument('-theano_backend', action="store", 
                    dest='theano_backend', type=bool, default=False)
parser.add_argument('-f')
args = parser.parse_args()
parser.print_help()
print('input args: ', args)

if args.theano_backend is True and args.quantize is True:
    raise ValueError("Quantize feature does not work with theano backend.")


usage: ipykernel_launcher.py [-h] [-input_fld INPUT_FLD]
                             [-output_fld OUTPUT_FLD]
                             [-input_model_file INPUT_MODEL_FILE]
                             [-output_model_file OUTPUT_MODEL_FILE]
                             [-output_graphdef_file OUTPUT_GRAPHDEF_FILE]
                             [-num_outputs NUM_OUTPUTS] [-graph_def GRAPH_DEF]
                             [-output_node_prefix OUTPUT_NODE_PREFIX]
                             [-quantize QUANTIZE]
                             [-theano_backend THEANO_BACKEND] [-f F]

set input arguments

optional arguments:
  -h, --help            show this help message and exit
  -input_fld INPUT_FLD
  -output_fld OUTPUT_FLD
  -input_model_file INPUT_MODEL_FILE
  -output_model_file OUTPUT_MODEL_FILE
  -output_graphdef_file OUTPUT_GRAPHDEF_FILE
  -num_outputs NUM_OUTPUTS
  -graph_def GRAPH_DEF
  -output_node_prefix OUTPUT_NODE_PREFIX
  -quantize QUANTIZE
  -theano_backend THEANO_BACKEND
 

initialize

In [23]:
from keras.models import load_model
import tensorflow as tf
from pathlib import Path
from keras import backend as K

output_fld =  args.input_fld if args.output_fld == '' else args.output_fld
if args.output_model_file == '':
    args.output_model_file = str(Path(args.input_model_file).name) + '.pb'
Path(output_fld).mkdir(parents=True, exist_ok=True)    
weight_file_path = str(Path(args.input_fld) / args.input_model_file)
print ('weight_file_path:',weight_file_path)

weight_file_path: weights_at_epoch_18.h5


Load keras model and rename output

In [24]:
from keras.models import model_from_json
K.set_learning_phase(0)
if args.theano_backend:
    K.set_image_data_format('channels_first')
else:
    K.set_image_data_format('channels_last')

try:
    with open("model.json",'r') as f:
        modelJSON = f.read()
        model = model_from_json(modelJSON)
        model.load_weights("weights_at_epoch_18.h5")
        print(model.summary())

except ValueError as err:
    print('''Input file specified ({}) only holds the weights, and not the model definition.
    Save the model using mode.save(filename.h5) which will contain the network architecture
    as well as its weights. 
    If the model is saved using model.save_weights(filename.h5), the model architecture is 
    expected to be saved separately in a json format and loaded prior to loading the weights.
    Check the keras documentation for more details (https://keras.io/getting-started/faq/)'''
          .format(weight_file_path))
    raise err
num_output = args.num_outputs
print ('num_output')
pred = [None]*num_output
pred_node_names = [None]*num_output
for i in range(num_output):
    pred_node_names[i] = args.output_node_prefix+str(i)
    pred[i] = tf.identity(model.outputs[i], name=pred_node_names[i])
print('output nodes names are: ', pred_node_names)

FileNotFoundError: [Errno 2] No such file or directory: 'model.json'

[optional] write graph definition in ascii

In [5]:
sess = K.get_session()

if args.graph_def:
    f = args.output_graphdef_file 
    tf.train.write_graph(sess.graph.as_graph_def(), output_fld, f, as_text=True)
    print('saved the graph definition in ascii format at: ', str(Path(output_fld) / f))

saved the graph definition in ascii format at:  model.ascii


convert variables to constants and save

In [6]:
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from tensorflow.tools.graph_transforms import TransformGraph #TransformGraph from Tensorflow does everything nicely

print (pred_node_names)
if args.quantize:
    transforms = ["quantize_weights", "quantize_nodes"]
    transformed_graph_def = TransformGraph(sess.graph.as_graph_def(), [], pred_node_names, transforms)
    constant_graph = graph_util.convert_variables_to_constants(sess, transformed_graph_def, pred_node_names) #variables that are frozen converted to constants Tensorflow
else:
    constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), pred_node_names)    
graph_io.write_graph(constant_graph, output_fld, args.output_model_file, as_text=False)
print('saved the freezed graph (ready for inference) at: ', str(Path(output_fld) / args.output_model_file))

['output_node0']
INFO:tensorflow:Froze 236 variables.
Converted 236 variables to const ops.
saved the freezed graph (ready for inference) at:  weights_at_epoch_18.h5.pb


In [7]:
"""Given a tensor flow model graph, how to find the input node and output node names??
Can also run it in the Terminal for fast checking
"""

'Given a tensor flow model graph, how to find the input node and output node names??\nCan also run it in the Terminal for fast checking\n'

In [26]:
import tensorflow as tf
from tensorflow.python.platform import gfile

GRAPH_PB_PATH = 'frozen_inference_graph.pb' #path to your .pb file
with tf.Session() as sess:
  print("load graph")
  with gfile.FastGFile(GRAPH_PB_PATH,'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    sess.graph.as_default()
    tf.import_graph_def(graph_def, name='')
    graph_nodes=[n for n in graph_def.node]
    
print (graph_nodes)

load graph


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [31]:
import tensorflow as tf
gf = tf.GraphDef()

filename = 'frozen_inference_graph.pb'
gf.ParseFromString(open(filename,'rb').read())

75785468

In [58]:
#print all output nodes
[n.name  +  '=>' + n.op for n in gf.node if n.op in ('Softmax','Placeholder')]

['image_tensor=>Placeholder']

In [59]:
#Check the number of nodes in Graph
print (len([n.name for n in gf.node]))

8525


In [60]:
print ([n.name for n in gf.node][1])

image_tensor


In [ ]:
#get the node by input name
#300x300

In [62]:
print ([n.name for n in gf.node][len([n.name for n in gf.node])-1])

num_detections


In [56]:
#output all the name of the nodes
for idx in range(0, len([n.name for n in gf.node])):
    print ([n.name for n in gf.node][idx])

Const
image_tensor
ToFloat
Preprocessor/map/Shape
Preprocessor/map/strided_slice/stack
Preprocessor/map/strided_slice/stack_1
Preprocessor/map/strided_slice/stack_2
Preprocessor/map/strided_slice
Preprocessor/map/TensorArray
Preprocessor/map/TensorArrayUnstack/Shape
Preprocessor/map/TensorArrayUnstack/strided_slice/stack
Preprocessor/map/TensorArrayUnstack/strided_slice/stack_1
Preprocessor/map/TensorArrayUnstack/strided_slice/stack_2
Preprocessor/map/TensorArrayUnstack/strided_slice
Preprocessor/map/TensorArrayUnstack/range/start
Preprocessor/map/TensorArrayUnstack/range/delta
Preprocessor/map/TensorArrayUnstack/range
Preprocessor/map/TensorArrayUnstack/TensorArrayScatter/TensorArrayScatterV3
Preprocessor/map/Const
Preprocessor/map/TensorArray_1
Preprocessor/map/TensorArray_2
Preprocessor/map/while/Enter
Preprocessor/map/while/Enter_1
Preprocessor/map/while/Enter_2
Preprocessor/map/while/Merge
Preprocessor/map/while/Merge_1
Preprocessor/map/while/Merge_2
Preprocessor/map/while/Less
Pr

FeatureExtractor/MobilenetV2/expanded_conv_2/depthwise/BatchNorm/batchnorm/add
FeatureExtractor/MobilenetV2/expanded_conv_2/depthwise/BatchNorm/batchnorm/Rsqrt
FeatureExtractor/MobilenetV2/expanded_conv_2/depthwise/BatchNorm/batchnorm/mul
FeatureExtractor/MobilenetV2/expanded_conv_2/depthwise/BatchNorm/batchnorm/mul_1
FeatureExtractor/MobilenetV2/expanded_conv_2/depthwise/BatchNorm/batchnorm/mul_2
FeatureExtractor/MobilenetV2/expanded_conv_2/depthwise/BatchNorm/batchnorm/sub
FeatureExtractor/MobilenetV2/expanded_conv_2/depthwise/BatchNorm/batchnorm/add_1
FeatureExtractor/MobilenetV2/expanded_conv_2/depthwise/Relu6
FeatureExtractor/MobilenetV2/expanded_conv_2/depthwise_output
FeatureExtractor/MobilenetV2/expanded_conv_2/project/weights
FeatureExtractor/MobilenetV2/expanded_conv_2/project/weights/read
FeatureExtractor/MobilenetV2/expanded_conv_2/project/Conv2D
FeatureExtractor/MobilenetV2/expanded_conv_2/project/BatchNorm/gamma
FeatureExtractor/MobilenetV2/expanded_conv_2/project/BatchNo

FeatureExtractor/MobilenetV2/expanded_conv_4/project/BatchNorm/batchnorm/Rsqrt
FeatureExtractor/MobilenetV2/expanded_conv_4/project/BatchNorm/batchnorm/mul
FeatureExtractor/MobilenetV2/expanded_conv_4/project/BatchNorm/batchnorm/mul_1
FeatureExtractor/MobilenetV2/expanded_conv_4/project/BatchNorm/batchnorm/mul_2
FeatureExtractor/MobilenetV2/expanded_conv_4/project/BatchNorm/batchnorm/sub
FeatureExtractor/MobilenetV2/expanded_conv_4/project/BatchNorm/batchnorm/add_1
FeatureExtractor/MobilenetV2/expanded_conv_4/project/Identity
FeatureExtractor/MobilenetV2/expanded_conv_4/add
FeatureExtractor/MobilenetV2/expanded_conv_4/output
FeatureExtractor/MobilenetV2/expanded_conv_5/input
FeatureExtractor/MobilenetV2/expanded_conv_5/expand/weights
FeatureExtractor/MobilenetV2/expanded_conv_5/expand/weights/read
FeatureExtractor/MobilenetV2/expanded_conv_5/expand/Conv2D
FeatureExtractor/MobilenetV2/expanded_conv_5/expand/BatchNorm/gamma
FeatureExtractor/MobilenetV2/expanded_conv_5/expand/BatchNorm/ga

FeatureExtractor/MobilenetV2/expanded_conv_7/expand/BatchNorm/gamma/read
FeatureExtractor/MobilenetV2/expanded_conv_7/expand/BatchNorm/beta
FeatureExtractor/MobilenetV2/expanded_conv_7/expand/BatchNorm/beta/read
FeatureExtractor/MobilenetV2/expanded_conv_7/expand/BatchNorm/moving_mean
FeatureExtractor/MobilenetV2/expanded_conv_7/expand/BatchNorm/moving_mean/read
FeatureExtractor/MobilenetV2/expanded_conv_7/expand/BatchNorm/moving_variance
FeatureExtractor/MobilenetV2/expanded_conv_7/expand/BatchNorm/moving_variance/read
FeatureExtractor/MobilenetV2/expanded_conv_7/expand/BatchNorm/batchnorm/add/y
FeatureExtractor/MobilenetV2/expanded_conv_7/expand/BatchNorm/batchnorm/add
FeatureExtractor/MobilenetV2/expanded_conv_7/expand/BatchNorm/batchnorm/Rsqrt
FeatureExtractor/MobilenetV2/expanded_conv_7/expand/BatchNorm/batchnorm/mul
FeatureExtractor/MobilenetV2/expanded_conv_7/expand/BatchNorm/batchnorm/mul_1
FeatureExtractor/MobilenetV2/expanded_conv_7/expand/BatchNorm/batchnorm/mul_2
FeatureExt

FeatureExtractor/MobilenetV2/expanded_conv_9/depthwise/BatchNorm/beta/read
FeatureExtractor/MobilenetV2/expanded_conv_9/depthwise/BatchNorm/moving_mean
FeatureExtractor/MobilenetV2/expanded_conv_9/depthwise/BatchNorm/moving_mean/read
FeatureExtractor/MobilenetV2/expanded_conv_9/depthwise/BatchNorm/moving_variance
FeatureExtractor/MobilenetV2/expanded_conv_9/depthwise/BatchNorm/moving_variance/read
FeatureExtractor/MobilenetV2/expanded_conv_9/depthwise/BatchNorm/batchnorm/add/y
FeatureExtractor/MobilenetV2/expanded_conv_9/depthwise/BatchNorm/batchnorm/add
FeatureExtractor/MobilenetV2/expanded_conv_9/depthwise/BatchNorm/batchnorm/Rsqrt
FeatureExtractor/MobilenetV2/expanded_conv_9/depthwise/BatchNorm/batchnorm/mul
FeatureExtractor/MobilenetV2/expanded_conv_9/depthwise/BatchNorm/batchnorm/mul_1
FeatureExtractor/MobilenetV2/expanded_conv_9/depthwise/BatchNorm/batchnorm/mul_2
FeatureExtractor/MobilenetV2/expanded_conv_9/depthwise/BatchNorm/batchnorm/sub
FeatureExtractor/MobilenetV2/expanded_

...

Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/FilterGreaterThan_95/Gather/Gather_2
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_95/split/split_dim
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_95/split
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_95/unstack
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_95/Minimum
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_95/Maximum
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_95/Minimum_1
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_95/Maximum_1
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_95/Mini

Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_97/concat
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_97/Area/split/split_dim
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_97/Area/split
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_97/Area/sub
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_97/Area/sub_1
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_97/Area/mul
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_97/Area/Squeeze
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_97/Greater/y
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_97/

Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_99/Area/sub
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_99/Area/sub_1
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_99/Area/mul
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_99/Area/Squeeze
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_99/Greater/y
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_99/Greater
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_99/Where
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_99/Reshape/shape
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/ClipToWindow_99/Reshape
Pos

Postprocessor/BatchMultiClassNonMaxSuppression/map/while/PadOrClipBoxList/cond_2/cond/concat
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/PadOrClipBoxList/cond_2/cond/concat/Switch
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/PadOrClipBoxList/cond_2/cond/sub/x
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/PadOrClipBoxList/cond_2/cond/sub
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/PadOrClipBoxList/cond_2/cond/sub/Switch
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/PadOrClipBoxList/cond_2/cond/ExpandDims/dim
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/PadOrClipBoxList/cond_2/cond/ExpandDims
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/PadOrClipBoxList/cond_2/cond/Merge
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/PadOrClipBoxList/cond_2/zeros/Const
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/PadOrClipBoxList/cond_2/zeros
Postprocessor/BatchMultiClassNonMaxSuppression/map/while/P

add/y
add
detection_boxes
detection_scores
detection_classes
num_detections


In [50]:
#optimised tensor for use great
#python -m tensorflow.python.tools.optimize_for_inference \
#  --input=tf_files/retrained_graph.pb \
#  --output=tf_files/optimized_graph.pb \
#  --input_names="input" \
#  --output_names="final_result"

In [70]:
output = filename + 'optimized_graph.pb'
print (output)

weights_at_epoch_18.h5.pboptimized_graph.pb


In [63]:
try:
    !python -m tensorflow.python.tools.optimize_for_inference --input=frozen_inference_graph.pb --output=optimized_graph.pb --input_names=image_tensor --output_names=num_detections

/home/oushesh/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
